# 3-Stage Clos Network to make Networks Scalable
###### <sup>Inputs: Bhavani Parise, Deepti Chandra; Developed by: Sarah Samuel</sup>
###### <sup>Web Documentation: https://developer.cisco.com/docs/sonic/#!3-stage-clos-network-with-static-vxlan-to-make-networks-scalable</sup>

Modern cloud-scale data center networks require increased server-to-server communication over a network that stays resilient despite the rapid increase in the number of devices.

A 3-stage Clos network interconnects data center network switches where each spine switch connects to all leaf switches. And each leaf switch connects to a server in the data center. Any server in the data center is just three hops away from another server. The first hop is from the server to the directly connected leaf switch, the second hop is across the spine switches to the destination leaf switch, and the third hop is between the destination leaf switch to the destination server. This network architecture is highly scalable. Also, irrespective of the number of devices in the data center, the number of hops between the servers or the end-hosts is always 3, ensuring consistent latency in the data center network.

The 3-stage Clos network is a robust IP-BGP underlay network for the data center. Over this network, you can configure overlay features such as Virtual Extensible Local Area Network (VXLAN).

Though you use generally switches in a 3-stage Clos network, this notebook demonstrates how to [Bring Up 3-Stage Clos Network as an Underlay Network](#bring-up-3-stage-clos-network-as-an-underlay-network) using Cisco 8000 series routers that run SONiC.

The following topology diagram depicts a simple 3-stage Clos network with two leaf routers in Tier-0 and two spine routers in Tier-1.



<center><img src="./images/522559.jpg" width="700"/></center>


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the live network nodes on the Cisco 8000 Emulator that runs in the background. The notebook refreshes the output of each cell-execution just beneath it.

After the topology is up, play through the following steps to [Bring Up 3 Stage Clos Network as an Underlay Network](#Bring-Up-3-Stage-Clos-Network-as-an-Underlay-Network):
* [Configure Host-Names](#Configure-Host-Names)
* [Assign IP-Addresses](#Assign-IP-Addresses)
* [Configure eBGP](#Configure-eBGP)
* [Verify BGP Route Exchange](#Verify-BGP-Route-Exchange)
* [Send Traffic from TREX](#Send-Traffic-from-TREX)
* [Verify Traffic Statistics](#Verify-Traffic-Statistics)

Finally, [clean up emulator session](#Clean-Up-Emulator-Session), once you are done.

### Bring up Topology

> Install prerequisite modules: This is required mainly for the first time you play this notebook. You can skip playing this cell for the subsequent runs.

In [1]:
%%capture cell-output
!sh prereq_install.sh

> Play the following cell to bring up topology and access ssh console of each device in the topology. This cell takes about 10 - 15 minutes to complete execution. When you see the **Sim status** displayed as  **{'localhost': 'running'}** underneath this cell, the topology is up. Wait for a few more seconds until the code accesses the ssh console of each device. 

> Avoid playing this cell more than once, without [cleaning up the emulator session](#Clean-Up-Emulator-Session).

In [1]:
from leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }

tb = access_device_consoles("leaf_spine.yaml", nodes)

3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 10-15 minutes.
Sim status:  {'localhost': 'running'}
Consoles can be reached by:

 S0 : telnet 172.17.0.7 44889

 S1 : telnet 172.17.0.7 41711

 L0 : telnet 172.17.0.7 33157

 L1 : telnet 172.17.0.7 46699

 trex : telnet 172.17.0.7 33278

*** Logging into the devices ***

2022-06-15 18:37:11,727: %UNICON-INFO: +++ S0 logfile /tmp/S0-cli-20220615T183711725.log +++

2022-06-15 18:37:11,728: %UNICON-INFO: +++ Unicon plugin linux (unicon.plugins.linux) +++

2022-06-15 18:37:11,738: %UNICON-INFO: +++ connection to spawn: ssh -l cisco 172.17.0.7 -p 60949, id: 140002679691920 +++

2022-06-15 18:37:11,742: %UNICON-INFO: connection to S0
The authenticity of host '[172.17.0.7]:60949 ([172.17.0.7]:60949)' can't be established.
ECDSA key fingerprint is SHA256:7AegQ+tId2Hs+dW/dfmoGiocsmkxurMwB/Nmd/AWP/I.
Are you sure you want to continue connecting (yes/no)

## Bring Up 3-Stage Clos Network as an Underlay Network

> To configure a 3-stage clos network, continue playing through the following steps:

### Configure Host-Names

> Configure host names for the spine and leaf routers for easy identification.

In [2]:
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')


2022-06-15 18:44:30,617: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config hostname SPINE0' +++
sudo config hostname SPINE0
Running command: service hostname-config restart
Reloading Monit configuration ...
Reinitializing monit daemon
Please note loaded setting will be lost after system reboot. To preserve setting, run `config save`.
cisco@sonic:~$ 

2022-06-15 18:44:32,054: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-06-15 18:44:33,259: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config hostname SPINE1' +++
sudo config hostname SPINE1
Running command: service hostname-config restart
Reloading Monit configuration ...
Reinitializing monit daemon
Please note loaded setting will be lost after system reboot. To preserve setting, run `config save`.
cisco@sonic:~$ 

2022-06-15 18:44

### Assign IP Addresses

> This step assigns IP addresses on the interfaces of all devices as per the topology diagram. And then saves the configurations.

In [3]:
out = nodes['S0'].execute('sudo config interface ip add Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add Ethernet1 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config interface ip add Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add Ethernet1 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config interface ip add Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet1 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet2 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config interface ip add Ethernet0 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet1 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet2 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')


2022-06-15 18:44:45,133: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Ethernet0 10.0.1.1/24' +++
sudo config interface ip add Ethernet0 10.0.1.1/24
cisco@sonic:~$ 

2022-06-15 18:44:46,006: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Ethernet1 10.0.2.1/24' +++
sudo config interface ip add Ethernet1 10.0.2.1/24
cisco@sonic:~$ 

2022-06-15 18:44:46,777: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Loopback0 10.10.10.100/32' +++
sudo config interface ip add Loopback0 10.10.10.100/32
cisco@sonic:~$ 

2022-06-15 18:44:47,573: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-06-15 18:44:48,629: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Ethernet0 10.0.3.1/

> Verify the configured IP addresses using the ```show ip interfaces``` command on the routers and ```ifconfig -a``` command on the TREX server.

In [4]:
for n in nodes:
   if (n == 'trex'):
      out = nodes[n].execute('ifconfig -a eth1')
      out = nodes[n].execute('ifconfig -a eth2')
   else:
      out = nodes[n].execute('show ip interfaces')  


2022-06-15 18:45:33,860: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show ip interfaces' +++
show ip interfaces
Interface    Master    IPv4 address/mask    Admin/Oper    BGP Neighbor    Neighbor IP
-----------  --------  -------------------  ------------  --------------  -------------
Ethernet0              10.0.1.1/24          up/up         N/A             N/A
Ethernet1              10.0.2.1/24          up/up         N/A             N/A
Loopback0              10.10.10.100/32      up/up         N/A             N/A
docker0                240.127.1.1/24       up/down       N/A             N/A
eth0                   192.168.122.215/24   up/up         N/A             N/A
lo                     127.0.0.1/16         up/up         N/A             N/A
cisco@sonic:~$ 

2022-06-15 18:45:35,303: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show ip interfaces' +++
show ip interfaces
Interface    Master    IPv4 address/mask    Admin/Oper    BGP Neighbor    Neighbor IP


### Configure eBGP

> The following section shows the eBGP (exterior Border Gateway Protocol) configuration on all spine and leaf routers. The spines are in Autonomous System (AS) 100 and the leaves in AS 200. 
> Note the usage of the ```allowas-in``` option in the ```neighbor``` command. The default behaviour of BGP is to reject learning routes from own AS. The ```allowas-in``` option overrides this default behavior and the nodes learn routes from own AS. 

In [5]:
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_0' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_1' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_0' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_1' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')


2022-06-15 18:45:45,135: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh -c 'configure terminal' -c 'hostname SPINE_0' -c 'router-id 10.10.10.100' -c 'router bgp 100' -c 'no bgp ebgp-requires-policy' -c 'neighbor 10.0.2.2 remote-as 200' -c 'neighbor 10.0.1.2 remote-as 200' -c 'address-family ipv4 unicast' -c 'neighbor 10.0.2.2 allowas-in' -c 'neighbor 10.0.1.2 allowas-in' -c 'network 10.0.1.0/24' -c 'network 10.0.2.0/24' -c 'network 10.10.10.100/32' -c 'redistribute connected'' +++
' -c 'redistribute connected'ress-family ipv4 unicast' -c 'neighbor 10.0.2.2 allowas-in' -c 'neighbor 10.0.1.2 allowas-in' -c 'network 10.0.1.0/24' -c 'network 10.0.2.0/24' -c 'network 10.10.10.100/32'
cisco@sonic:~$ 

2022-06-15 18:45:45,752: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh -c 'configure terminal' -c 'hostname SPINE_1' -c 'router-id 10.10.11.100' -c 'router bgp 100' -c 'no bgp ebgp-requires-policy' -c 'neighbor 10.0.3.2 remote-as 200' -c 'neighbor 10.0.4.2 

> Execute the command ```show ip bgp``` on all tier-0 and tier-1 routers to ensure eBGP is up.

In [6]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''vtysh \
      -c 'show ip bgp'
      ''')


2022-06-15 18:45:50,572: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh       -c 'show ip bgp'' +++
vtysh       -c 'show ip bgp'
BGP table version is 13, local router ID is 10.10.10.100, vrf id 0
Default local pref 100, local AS 100
Status codes:  s suppressed, d damped, h history, * valid, > best, = multipath,
               i internal, r RIB-failure, S Stale, R Removed
Nexthop codes: @NNN nexthop's vrf id, < announce-nh-self
Origin codes:  i - IGP, e - EGP, ? - incomplete

   Network          Next Hop            Metric LocPrf Weight Path
*  10.0.1.0/24      10.0.2.2                               0 200 100 i
*                   10.0.1.2                 0             0 200 i
*                   0.0.0.0                  0         32768 ?
*>                  0.0.0.0                  0         32768 i
*  10.0.2.0/24      10.0.2.2                 0             0 200 i
*                   10.0.1.2                               0 200 100 i
*                   0.0.0.0      

> This section checks the routes learnt through BGP by executing the command ```show ip route``` on all nodes in tier-0 and tier-1. The lines starting with the letter **B** are the routes learnt through BGP.

In [7]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''vtysh \
      -c 'show ip route'
      ''')


2022-06-15 18:45:59,436: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh       -c 'show ip route'' +++
vtysh       -c 'show ip route'
Codes: K - kernel route, C - connected, S - static, R - RIP,
       O - OSPF, I - IS-IS, B - BGP, E - EIGRP, N - NHRP,
       T - Table, v - VNC, V - VNC-Direct, A - Babel, D - SHARP,
       F - PBR, f - OpenFabric,
       > - selected route, * - FIB route, q - queued, r - rejected, b - backup

K>* 0.0.0.0/0 [0/202] via 192.168.122.1, eth0, 00:10:12
C>* 10.0.1.0/24 is directly connected, Ethernet0, 00:01:14
C>* 10.0.2.0/24 is directly connected, Ethernet1, 00:01:13
B>* 10.0.3.0/24 [20/0] via 10.0.1.2, Ethernet0, weight 1, 00:00:12
B>* 10.0.4.0/24 [20/0] via 10.0.2.2, Ethernet1, weight 1, 00:00:10
B>* 10.0.5.0/24 [20/0] via 10.0.1.2, Ethernet0, weight 1, 00:00:12
B>* 10.0.6.0/24 [20/0] via 10.0.2.2, Ethernet1, weight 1, 00:00:10
C>* 10.10.10.100/32 is directly connected, Loopback0, 00:01:13
B>* 10.10.10.200/32 [20/0] via 10.0.1.2, Ethern

> Ping IP addresses on the LEAF0 (L0) router from the LEAF1 (L1) router and vice-versa. This ensures that the 3-stage Clos network is ready for end-to-end traffic flow.

In [8]:
out = nodes['L1'].execute('ping -c5 10.0.5.1')
out = nodes['L1'].execute('ping -c5 10.10.10.200')
out = nodes['L0'].execute('ping -c5 10.0.6.1')
out = nodes['L0'].execute('ping -c5 10.10.11.200')


2022-06-15 18:46:05,789: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.0.5.1' +++
ping -c5 10.0.5.1
PING 10.0.5.1 (10.0.5.1) 56(84) bytes of data.
64 bytes from 10.0.5.1: icmp_seq=1 ttl=63 time=21.2 ms
64 bytes from 10.0.5.1: icmp_seq=2 ttl=63 time=12.2 ms
64 bytes from 10.0.5.1: icmp_seq=3 ttl=63 time=14.3 ms
64 bytes from 10.0.5.1: icmp_seq=4 ttl=63 time=15.3 ms
64 bytes from 10.0.5.1: icmp_seq=5 ttl=63 time=21.9 ms

--- 10.0.5.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 10ms
rtt min/avg/max/mdev = 12.161/16.954/21.896/3.892 ms
cisco@sonic:~$ 

2022-06-15 18:46:09,969: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.10.10.200' +++
ping -c5 10.10.10.200
PING 10.10.10.200 (10.10.10.200) 56(84) bytes of data.
64 bytes from 10.10.10.200: icmp_seq=1 ttl=63 time=43.3 ms
64 bytes from 10.10.10.200: icmp_seq=2 ttl=63 time=16.6 ms
64 bytes from 10.10.10.200: icmp_seq=3 ttl=63 time=17.7 ms
64 bytes from 10.10.10

**Pinging to remote leaf ip address is successful and so the 3-stage Clos network is up and ready for data traffic**

### Send Traffic from TREX

> [TREX](https://trex-tgn.cisco.com/trex/doc/trex_manual.html#_introduction) is a software traffic generator that runs on Linux. To simulate server-to-server traffic flow across a 3-stage Clos network, connect the TREX software traffic generator ports as the end-hosts to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<center><img src="./images/3clos-traffic.png" width="700"/></center>

> The function ```generate_bidir_traffic``` generates a bidirectional traffic burst for 1 second. After the cell execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [9]:
trex_ipaddress = str(ports['trex']['HostAgent'])
trex_port = str(ports['trex']['xr_redir22'])

generate_bidir_traffic(trex_ipaddress, trex_port)

Last login: Wed Jun 15 18:37:13 2022 from gateway
[root@localhost ~]# ifconfig eth1 up; ifconfig eth2 up
[root@localhost ~]# cd /opt/cisco/trex/latest/cap2/; ls -lart test-new.yaml; cd  /opt/cisco/trex/latest/
-rw-r--r-- 1 root root 556 Jun 15 18:46 test-new.yaml
[root@localhost latest]# ./t-rex-64 -f cap2/test-new.yaml -m 300 -d 1
exit
Trying to bind to vfio-pci ...
Trying to compile and bind to igb_uio ...
ERROR: We don't have precompiled igb_uio.ko module for your kernel version.
Will try compiling automatically...
Success.

/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.98 please wait  ... 
 set driver name net_virtio 
 driver capability  : SLRO 
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec .
port : 0 
------------
link         :  link : Link Up - speed 100000 Mbps - full-duplex
promiscuous  : 0 
port : 1 
------------
link         :  link : Link 

### Verify Traffic Statistics

> Check the interface counters on the nodes to ensure that the traffic is sent and received as seen in the traffic generator output.
    
> Note: "show interface counters rif" displaying 0s for all fields, is a known issue on the emulator. Users will experience expected behaviours on the physical setup.


In [10]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('show interface counters rif')


2022-06-15 18:47:01,456: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface counters rif' +++
show interface counters rif
    IFACE    RX_OK    RX_BPS    RX_PPS    RX_ERR    TX_OK    TX_BPS    TX_PPS    TX_ERR
---------  -------  --------  --------  --------  -------  --------  --------  --------
Ethernet0        0  0.00 B/s    0.00/s         0        0  0.00 B/s    0.00/s         0
Ethernet1        0  0.00 B/s    0.00/s         0        0  0.00 B/s    0.00/s         0
cisco@sonic:~$ 

2022-06-15 18:47:02,570: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface counters rif' +++
show interface counters rif
    IFACE    RX_OK    RX_BPS    RX_PPS    RX_ERR    TX_OK    TX_BPS    TX_PPS    TX_ERR
---------  -------  --------  --------  --------  -------  --------  --------  --------
Ethernet0        0  0.00 B/s    0.00/s         0        0  0.00 B/s    0.00/s         0
Ethernet1        0  0.00 B/s    0.00/s         0        0  0.00 B/s    0.00/

> You have now successfully brought up a simple 3-stage Clos network, sent traffic across it. This forms the IP-BGP underlay for your data center network. 

> Scale up these configurations as per the number of devices in your data centre. 

### Clean Up Emulator Session

> Bring down the emulator after you are done. This cell does not produce any output and it takes about a minute to complete.

In [12]:
# Close the sim
sim.stop()
sim.clean()
shutil.rmtree(sim_dir)

In [38]:
!rm -rf *.json

> Do let us know of your feedback or queries about this notebook at mig-notebooks@cisco.com.